In [3]:
import pandas as pd
import numpy as np
# LDA, tSNE
from sklearn.manifold import TSNE
from gensim.models.ldamodel import LdaModel
# NLTK
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re
# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
# Visualization
import matplotlib
output_notebook()

C:\Users\rober\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Loading BokehJS ...

In [4]:
#data to 

In [40]:
with open('MovieSummaries\plot_summaries.txt', 'r', encoding="utf8") as myfile:
    summary = []
    titles = []
    count = 0
    for row in myfile:
        temp = row.split('\t')
        titles.append(temp[0])
        summary.append(temp[1])
        count += 1
        if count%1000 == 0:
            print(count)
    #summary=myfile.readlines()
#with open('movie_titles_bottom_250.txt', 'r') as myfile:
#    title=myfile.readlines()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000


In [41]:
import csv
title = ["" for _ in range(42306)]
year = [2019 for _ in range(42306)]
with open('MovieSummaries\movie.metadata.tsv','r', encoding="utf8") as tsvin:
    tsvin = csv.reader(tsvin, delimiter='\t')
    c = 0
    for row in tsvin:
        if row[0] in titles:
            i = titles.index(row[0])
            title[int(i)] = row[2]
            #print(row[3][:4])
            try:
                tempyear = int(row[3][0:4])
                if tempyear > 1900:
                    year[int(i)] = tempyear
            except:
                pass
        c += 1
        if c%1000 == 0:
            print(c)
        

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000


In [27]:
print(len(summary))
nltk.download('punkt')

42306
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import string
spc = 0
def strip_proppers(text):
    global spc
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    spc += 1
    if (spc%1000 == 0):
        print(spc)
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
preprocess = [strip_proppers(doc) for doc in summary]


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000


In [29]:
df = pd.DataFrame(columns=['Title','Year','Summary'])
se = pd.Series(preprocess)
ti = pd.Series(title)
ye = pd.Series(year)
df['Summary'] = se.values
df['Title'] = ti.values
df['Year'] = ye.values

In [30]:
# Removing numerals:
df['Summary_tokens'] = df.Summary.map(lambda x: re.sub(r'\d+', '', x))
# Lower case:
df['Summary_tokens'] = df.Summary.map(lambda x: x.lower())
print(df['Summary_tokens'][0][:500])
df.to_csv('Combine_bottom.csv', encoding= 'utf-8')

a hard-working taxi driver and a saxophonist develop a bizarre love-hate relationship and despite their prejudices realize they are n't so different after all


In [31]:
df['Summary_tokens'] = df.Summary_tokens.map(lambda x: RegexpTokenizer(r'\w+').tokenize(x))
print(df['Summary_tokens'][0][:25])

['a', 'hard', 'working', 'taxi', 'driver', 'and', 'a', 'saxophonist', 'develop', 'a', 'bizarre', 'love', 'hate', 'relationship', 'and', 'despite', 'their', 'prejudices', 'realize', 'they', 'are', 'n', 't', 'so', 'different']


In [32]:
snowball = SnowballStemmer("english")  
df['Summary_tokens'] = df.Summary_tokens.map(lambda x: [snowball.stem(token) for token in x])
print(df['Summary_tokens'][0][:25])

['a', 'hard', 'work', 'taxi', 'driver', 'and', 'a', 'saxophonist', 'develop', 'a', 'bizarr', 'love', 'hate', 'relationship', 'and', 'despit', 'their', 'prejudic', 'realiz', 'they', 'are', 'n', 't', 'so', 'differ']


In [33]:
nltk.download('stopwords')
stop_en = stopwords.words('english')
df['Summary_tokens'] = df.Summary_tokens.map(lambda x: [t for t in x if t not in stop_en]) 
print(df['Summary_tokens'][0][:25])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['hard', 'work', 'taxi', 'driver', 'saxophonist', 'develop', 'bizarr', 'love', 'hate', 'relationship', 'despit', 'prejudic', 'realiz', 'n', 'differ']


In [34]:
df['Summary_tokens'] = df.Summary_tokens.map(lambda x: [t for t in x if len(t) > 1])
print(df['Summary_tokens'][0][:25])

['hard', 'work', 'taxi', 'driver', 'saxophonist', 'develop', 'bizarr', 'love', 'hate', 'relationship', 'despit', 'prejudic', 'realiz', 'differ']


In [35]:
from gensim import corpora, models
np.random.seed(2017)
texts = df['Summary_tokens'].values
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = models.ldamodel.LdaModel(corpus, id2word=dictionary, 
                                    num_topics=5, passes=5, minimum_probability=0)

In [36]:
hm = np.array([[y for (x,y) in ldamodel[corpus[i]]] for i in range(len(corpus))])


In [37]:
tsne = TSNE(random_state=2017, perplexity=30)
tsne_embedding = tsne.fit_transform(hm)
tsne_embedding = pd.DataFrame(tsne_embedding, columns=['x','y'])
tsne_embedding['hue'] = hm.argmax(axis=1)



In [38]:
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
import bokeh
bokeh.io.reset_output()
source = ColumnDataSource(
        data=dict(
            x = tsne_embedding.x,
            y = tsne_embedding.y,
            colors = [all_palettes['Set1'][8][i] for i in tsne_embedding.hue],
            title = df.Title,
            year=df.Year,
            alpha = [0.9] * tsne_embedding.shape[0],
            size = [7] * tsne_embedding.shape[0]
        )
    )
hover_tsne = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Title:</span>
            <span style="font-size: 12px">@title</span>

        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=1000, plot_height=500, tools=tools_tsne, title='Movies')
plot_tsne.circle('x', 'y', size='size', fill_color='colors',
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, name="df")

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    title = data['title']
    year = data['year']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        if (year[i] <= f) {
            alpha[i] = 0.9
            size[i] = 7
        } else {
            alpha[i] = 0.05
            size[i] = 4
        }
    }
    source.trigger('change');
""")


slider = Slider(start=df.Year.min(), end=df.Year.max(), value=0, step=1, title="Before year: ")
slider.js_on_change('value', callback)

layout = column(slider, plot_tsne)

In [39]:

show(layout)


In [25]:
from bokeh.resources import CDN
from bokeh.embed import file_html

#https://www.kaggle.com/yohanb/lda-visualized-using-t-sne-and-bokeh
html = file_html(plot_tsne, CDN, "tsne_cmu")
htmlfile = open("tsne_cmu5_revenue.html", "w")
htmlfile.write(html)
htmlfile.close()